In [1]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [3]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'title': '東京駅(東京都)周辺のイベント',
   'content': '開催中 2025年3月20日(祝)～6月3日(火) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都) CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ) *   [美術展・博物展](https://www.walkerplus.com/event_list/eg0107/) 開催中 2025年3月20日(祝)～6月3日(火) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都) CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ) *   [グルメ・フードフェス](https://www.walkerplus.com/event_list/eg0117/) *   ![Image 6](https://www.walkerplus.com/asset/images/common/ico_charge.svg) 終了間近 2025年4月1日(火)～5月31日(土) 東京駅(東京都), 二重橋前駅(東京都), 京橋駅(東京都), 大手町駅(東京都), 有楽町駅(東京都) *   [グルメ・フードフェス](https://www.walkerplus.com/event_list/eg0117/) *   ![Image 8](https://www.walkerplus.com/asset/images/common/ico_parking.svg) 2025年4月5日(土)～6月15日(日) 東京駅(東京都), 二重橋前駅(東京都), 大手町駅(東京都), 日本橋駅(東京都), 京橋駅(東京都) *   [美術展・博物展](https://www.walkerplus.com/event_list/eg0107/) 2025年3月1日(土)～6月1日(日) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 東京駅(東京都), 銀座一丁目

In [4]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [5]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [6]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [7]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [8]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response(question, tools)
print(response_message)

東京都と沖縄県の面積を比較すると、沖縄県の方が広いです。具体的には、沖縄県の面積は約2,276.64 km²、東京都の面積は約2,194.05 km²です。したがって、沖縄県が東京都よりも広いことがわかります。


In [9]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

最近1ヶ月以内の東京駅周辺でのイベント情報をいくつかご紹介します。

1. **SNOOPY Loves NATURE**
   - **期間**: 2023年9月22日（金）〜10月5日（木）
   - **場所**: 東京駅一番街
   - **内容**: スヌーピーをテーマにした特別イベントが開催されます。
   - [詳細はこちら](https://town.snoopy.co.jp/event/snoopy_loves_nature_tokyo2023.html)

2. **東京の秋のイベント**
   - **期間**: 2023年9月15日（金）〜9月17日（日）
   - **内容**: 赤穂浪士まつりなど、さまざまな秋のイベントが行われます。
   - [詳細はこちら](https://activityjapan.com/feature/tokyo-fall-event/)

3. **東京近郊の注目イベント**
   - **内容**: 注目の発表や花火大会、秋祭りなどが行われる予定です。
   - [詳細はこちら](https://trip.iko-yo.net/articles/1529)

これらのイベントは家族連れや観光客にも楽しめる内容が盛りだくさんですので、ぜひ参加してみてください！


In [10]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

'質問:こんにちは！'

こんにちは！どういったことをお手伝いできますか？


'質問:東北6県は？'

東北地方には6つの県があります。以下がその県名です：

1. 福島県（ふくしまけん）
2. 宮城県（みやぎけん）
3. 秋田県（あきたけん）
4. 岩手県（いわてけん）
5. 山形県（やまがたけん）
6. 青森県（あおもりけん）

これらの県は、自然豊かで文化的にも多様な地域です。


'質問:宮城県のお土産について検索した結果を教えて'

宮城県のお土産として人気のある品々についての情報をいくつかご紹介します。

1. **牛たん** - 宮城県は牛たんで有名で、特に仙台名物として知られています。様々な加工品が販売されているほか、レストランで新鮮な牛たんを楽しむこともできます。

2. **笹かまぼこ** - 宮城の伝統的な魚の練り物で、ふわふわとした食感が特徴です。お土産用に個包装されている商品もあります。

3. **ずんだ餅** - エダマメをすりつぶして作ったずんだを使ったお餅のスイーツで、甘さと独特の風味が楽しめます。

4. **仙台味噌** - 地元の特産品の一つで、家庭料理にも使用される人気の調味料です。

5. **萩の月** - 宮城県名物の和菓子で、餡（あん）が入った洋風の饅頭。見た目も美しく、贈り物にぴったりです。

6. **お米** - 宮城県産の美味しいお米もお土産として人気があります。特に「ひとめぼれ」など、種類が豊富です。

7. **宮城の日本酒** - 地元の酒蔵で造られた日本酒も、お土産におすすめです。特に、地酒はその土地の風味を楽しめます。

これらのお土産は、宮城県内の観光地や空港、駅などで手に入れることができます。また、さらに詳しい情報や特産品の紹介については、以下のリンクを参考にしてください。

- [宮城県のお土産 51選！](https://atlas-log.com/miyagi-souvenir/)
- [宮城で買うお土産20選](https://tsplus.asahi.co.jp/articles/gift/75865/)
- [仙台の方のオススメお土産22選](https://travel.rakuten.co.jp/mytrip/howto/sendai-souvenir) 

ぜひ、宮城県を訪れた際には立ち寄ってみてください。

---ご利用ありがとうございました！---
